# Are users home locations in Vulnerable Locations?

In [1]:
import pandas as pd; import numpy as np; from multiprocessing import Pool, Manager; import numpy as np; import geopandas as gpd
import matplotlib.pyplot as plt; import seaborn as sns
import matplotlib, os, json, sys, time, datetime
from bson import json_util

In [2]:
input_directory = "/data/chime/geo/sandy_new_jersey_coastline_users_stage2"
output_directory = "/data/chime/geo/sandy_new_jersey_coastline_users_geovulnerable"

#### Import the ZoneA Geometry

In [3]:
import fiona, shapely; from osgeo import ogr; from shapely.geometry import mapping, shape
c = fiona.open('../EvacuationZones/New_Jersey/coastline.shp','r')
pol = c.next(); zoneA = shape(pol['geometry']).buffer(0); zoneA.is_valid

True

#### Import the user metadata DF (Phasing this out)

In [4]:
_user_meta = pd.read_json(input_directory+'/temporal_clustered_user_meta.json')

In [5]:
_user_meta = _user_meta.sort_index()
_user_meta.head(3)

,home_cluster,home_cluster_coords,tweets,uid,user
0,1.0,"{""type"": ""Point"", ""coordinates"": [-73.69255456...",14231,75153082,Andrewthemark
1,2.0,"{""type"": ""Point"", ""coordinates"": [-74.40503166...",3857,318103926,lanceeryann
2,1.0,"{""type"": ""Point"", ""coordinates"": [-73.81305625...",3778,190488649,giaco1025


## 0. Import all of the individual user dataframes

In [6]:
users_in = sorted(os.listdir(input_directory))
users_in = [x for x in users_in if x != "temporal_clustered_user_meta.json"]
print("Found {0} users in {1}".format(len(users_in), input_directory))

Found 1160 users in /data/chime/geo/sandy_new_jersey_coastline_users_stage2


In [7]:
def loader_function(args):
    uFile, path, q = args
    u = json.load(open(path+"/"+uFile,'r'))
    tweets = []
    for t in u['features']:
        if t['geometry']:
            t['properties']['geometry'] = shape(t['geometry'])
        t['properties']['date'] = pd.Timestamp(t['properties']['date'])
        tweets.append(t['properties'])
    q.put(1)
    return gpd.GeoDataFrame(tweets)

In [8]:
#Parallel runtime
p = Pool(30)
m = Manager()
q = m.Queue()

args = [(i, input_directory, q) for i in users_in]
result = p.map_async(loader_function, args)

# monitor loop
while True:
    if result.ready():
        break
    else:
        size = q.qsize()
        sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(size, size/len(args)*100))
        time.sleep(0.5)
sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(q.qsize(), q.qsize()/len(args)*100))
users = result.get()
p.close()

Processed: 1160, 100%

In [9]:
users.sort( key=lambda x: len(x), reverse=True ) 

`users` is an array of user dataframes. Now find which users have _home locations_ in Zone A

In [10]:
x = users[100]
print(len(x))
x.head(1)

684


,cluster,cluster_center,coords,date,day_cluster,geo_delta,geometry,home_cluster_id,speed,text,time_delta,tweet_id,uid,user
0,1,"{""type"": ""Point"", ""coordinates"": [-74.42105253...","[-74.42105253, 39.37529605]",2012-09-01 16:11:17+00:00,9,NaN,POINT (-74.42105253 39.37529605),5.0,NaN,Out to eat with my grandmom (:,NaN,241931239983046656,265033252,Hottoddy___


In [11]:
def get_home_cluster_center(userDF):
    return shape(json.loads(userDF.query('cluster=='+str(userDF.home_cluster_id.values[0])).cluster_center.values[0]))

get_home_cluster_center(users[0]).within(zoneA)

False

## 1. Determine who's home cluster center is in Zone A

In [12]:
def insideZoneA(p):
    if p==None:
        return False
    else:
        return p.within(zoneA)

In [13]:
vuln = []
non_vuln = []
for idx, u in enumerate(users):
    if (insideZoneA(get_home_cluster_center(u))):
        vuln.append(u)
    else:
        non_vuln.append(u)
    sys.stderr.write("\r"+str(idx+1))
sys.stderr.write("\rDone...")
sys.stderr.write("Identified {0} vulnerable users and {1} non-vulnerable".format(len(vuln),len(non_vuln)))

Done...Identified 356 vulnerable users and 804 non-vulnerable

In [14]:
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

## 2. Write out just the GeoVulnerable, just in case we need them for something later

In [15]:
def safe_mapping(p):
    if p==None or np.isnan(p).any():
        return None
    else:
        return mapping(p)
def safe_json_export(args):
    df, path = args
    df = df.copy()
    uName = df.head(1).user.values[0].lower()
    df['date'] = df['date'].apply(lambda t: datetime.datetime.strftime(t,'%Y-%m-%dT%H:%M:%SZ'))

    clean = df.where((pd.notnull(df)), None)
    geojson = {"type":"FeatureCollection","features":[]}
    for _, row in clean.iterrows():
        geom = safe_mapping(row.geometry)
        feature = {'type':'Feature',
                   'geometry':geom,
                   'properties':row.to_dict()
                    }
        del feature['properties']['geometry']
        geojson['features'].append(feature)
    
    with open(path+"/"+uName+'.geojson','w') as oFile:
        json.dump(geojson, oFile) 

In [16]:
for idx, u in enumerate(vuln):
    safe_json_export((u,output_directory))
    sys.stderr.write("\r{0} processed".format(idx+1))

356 processed

# 3. Prepare for Analysis

All of these users should already exist in a format prepared for visualizing, these users can be pasted into Google Sheets

In [17]:
for uName in sorted([u.user[0].lower() for u in vuln]):
    print(uName)

08243
1stfitnessmc
_asvplady
_daniellevaldez
aclive35
adtwenty7
aimee_sh
alanjonesbeats
alex_florke
alexrosenkranz
alreadywifedup1
alyssagabrielaa
amandaatar
amandastilts
amberaalonzo
ameliaianiro
americabheleem
amkowski11
amyy1026
amyywalshh
angie_was_heree
anniexdarling
appiemallous
aprilydavis
arozcoy
aschwerzy
asianbrizl3
ayo_tahj
b_bott84
b_mazzz
badblasian
barbershairport
bbygrl_xoxo
bbytripk
beanie5469
bellax3
belmardays
benirvine
billyhoward423
bkhomesteader
bleedblue0415
brian_perks
brii_fi
brittni_linn
brooklynbridgex
broxlan
brucknerchase
brycenihill
c_it_my_way
caittfoley
calllmejas_
camposr1979
carleylarrison
caseadilia
catymogo
chandlernichols
cheerbowsndhoes
chettyoubetcha
chippiesue
chris22_steve
chrisalicious85
chrissie115
chrisstineey
chuck430
chugggss
claire_reffitt
cmccluskey18
coachhollywoodp
coastalartists
cookecapemay
coolinlike_
coolkym
courtneyhabick
cravejams
creamstaffa
cwalks3
cybernjm
d_fitz_187
daddyjerzey
damageathletics
daniellemillen1
darrianrojas
debha

In [18]:
len(vuln)

356

# Create rules file from UID

In [19]:
rules = []
these_rules = []
for idx,u in enumerate(vuln):

    rule = "from:"+u.uid[0]
    these_rules.append(rule)
    
    if idx%25==0 and idx>0:
        rules.append(" OR ".join(these_rules))
        these_rules = []

output = []
for r in rules:
    output.append({"value":r})

with open('../../GNIP/Sandy/NJ_GeoVulnerable_Contextual/rules.json','w') as oFile:
    json.dump(output, oFile)